In [1]:
import pandas as pd

from external_lib.NFLlib.score import NFLAssignmentScorer, check_submission
from external_lib.NFLlib.features import add_track_features

from lib.math.coodinate import generate_rotation_maxrix
from lib.math.coodinate import convert_3d_to_2d

from lib.noglobal import noglobal

import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random

In [2]:
n_debug_samples = 5
random_state = 4
CONF_THRE = 0.3
max_iter = 1000
DIG_STEP = 3
DIG_MAX = 30
base_dir = "/work/data/input/nfl-health-and-safety-helmet-assignment"

debug = True


paramater = {};
paramater["n_debug_samples"] = n_debug_samples
paramater["random_state"] = random_state
paramater["CONF_THRE"] = CONF_THRE
paramater["max_iter"] = max_iter
paramater["DIG_STEP"] = DIG_STEP
paramater["DIG_MAX"] = DIG_MAX

## Target label

In [3]:
#taret_csv = "baseline/all_result_modifyed.csv"
#predicted_df =  pd.read_csv(taret_csv)
#predicted_df["zone"] = predicted_df["key"].apply(lambda x:x.split("_")[2])

#predicted_df[predicted_df["key"].str.contains(target_view)].sort_values("frame")

In [4]:
target_view = "57583_000082_Sideline"

In [5]:
labels = pd.read_csv(f"{base_dir}/train_labels.csv")

if debug:
    tracking = pd.read_csv(f'{base_dir}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{base_dir}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/test_baseline_helmets.csv')
    
tracking = add_track_features(tracking)
tracking["z"] = 6.0


view_mode = "Endzone"
labels = labels[labels["video_frame"].str.contains(view_mode)]
tracking = tracking
helmets = helmets[helmets["video_frame"].str.contains(view_mode)]


#labels = labels[labels["video_frame"].str.contains(target_view)]
#tracking = tracking[tracking["game_play"] ==  "_".join(target_view.split("_")[:2])]
#helmets = helmets[helmets["video_frame"].str.contains(target_view)]
    
    
    
def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if 'video' not in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df



if debug:
    helmets = add_cols(helmets)
    labels = add_cols(labels)    
    
    # Select `n_debug_samples` worth of videos to debug with
    sample_videos = labels['video'].drop_duplicates() \
        .sample(n_debug_samples, random_state=random_state).tolist()
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]

    
helmets["frame"] = helmets["video_frame"].apply(lambda x:x.split("_")[3])
helmets["video_frame_without_position"] = helmets["video_frame"].apply(lambda x: "_".join( x.split("_")[:2] + [x.split("_")[3]]   )  )
    
for key,hel_each_df in helmets.groupby("game_play"):    
    min_frame = hel_each_df["frame"].astype(int).min()
    max_frame = hel_each_df["frame"].astype(int).max()
    tracking = tracking[tracking["est_frame"] > (min_frame-5)]
    tracking = tracking[tracking["est_frame"] < max_frame+5]  


In [6]:
@noglobal()
def find_nearest(array, value):
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


### [0,1]ではないけど、正規化
@noglobal()
def norm_arr(a):
    a = a-a.min();
    a = a/(a.max());
    return a
    
@noglobal()
def dist(a1,a2):
    return np.linalg.norm(a1-a2);


@noglobal()
def dist_for_different_len(a1,a2,max_iter):
    assert len(a1) >= len(a2), f'{len(a1)}, {len(a2)}'
    
    len_diff = len(a1) - len(a2);

    a2 = norm_arr(a2);
    
    if len_diff == 0:
        a1 = norm_arr(a1);
        return dist(a1,a2),();
    else:
        min_dist = 10000;
        min_delete_idx = None;
        cnt = 0;
        
        del_list = list(itertools.combinations(range(len(a1)),len_diff));    
        
        ## 組み合わせの総数が指定した数より大きい場合、ランダムサンプリングする。。
        if (len(del_list) > max_iter):
            del_list = random.sample(del_list,max_iter);
            
        for delete_idx in del_list:
            this_a1 = np.delete(a1,delete_idx);
            this_a1 = norm_arr(this_a1);
            this_dist = dist(this_a1,a2);
            
            if (min_dist > this_dist):
                min_dist = this_dist
                min_delete_idx = delete_idx;
            
            
        return min_dist,min_delete_idx


@noglobal()
def rotate_arr(u,view, t, isDeg=True):
    if isDeg:
        t = np.deg2rad(t);
        
    if(view == "Endzone"):
        from_base_point = np.array([150,0,6])
        target_axis = "y"
        to_Point = np.array([60,0,5])    
        length_vector = from_base_point - to_Point                            
        rotation_matrix = generate_rotation_maxrix(-1*t,axis=target_axis,isDeg=False);
        from_point = to_Point + rotation_matrix@length_vector
        
        
    elif(view == "Sideline"):
        to_Point = np.array([60,53.3/2,0])
        from_point = [60 + t,71,40]        
    else:
        raise Exception(f"You can specify only Endzone or Sideline, not {view}");
    
    
    
    return convert_3d_to_2d(u[["x","y","z"]],from_point=from_point,to_point=to_Point)

    

    
@noglobal(excepts = ["rotate_arr"])
def dist_rot(tracking_df, a2,view,DIG_MAX,DIG_STEP,max_iter):
    tracking_df = tracking_df.sort_values('x')
    x = tracking_df['x']
    y = tracking_df['y']
    min_dist = 10000
    min_idx = None
    min_x = None
    
    best_angle = 0;
    
    
    for dig in range(-1*DIG_MAX,DIG_MAX+1,DIG_STEP):                
        
        if (view == "Endzone"):
        
            if (dig <0):
                arr = rotate_arr(tracking_df,view,180 + dig);
            else:
                arr = rotate_arr(tracking_df,view,dig);
                
            arr_df = arr
            arr = arr[["vertical","horizon"]].values.T
        elif (view == "Sideline"):
            arr = rotate_arr(tracking_df,view,dig)
            arr_df = arr
            arr = arr[["horizon","depth"]].values.T
        else:
            raise Exception(f"unknown view mode is specified: {view}")
                                    

        ##  a2: ヘルメットの座標        
        this_dist, this_idx = dist_for_different_len(np.sort(arr[0]), a2,max_iter)
        if min_dist > this_dist:
            min_dist = this_dist
            min_idx = this_idx
            min_x = arr[0]
            best_angle = dig
            best_arr_df = arr_df
            
        
    #display(arr_df)
    tracking_df['x_rot'] = min_x
    player_arr = tracking_df.sort_values('x_rot')['player'].values
    players = np.delete(player_arr,min_idx)
    
    
    return min_dist, players,best_angle  
    

@noglobal()
def mapping_df(args):
    
    ##########################
    ### data difinition ### 
    video_frame, helmets_df = args[:2]
    tracking = args[2]
    parameter = args[3]    
    CONF_THRE = parameter["CONF_THRE"];
    DIG_STEP = parameter["DIG_STEP"];
    DIG_MAX = parameter["DIG_MAX"];
    
    max_iter = parameter["max_iter"];
            
    gameKey,playID,view,frame = video_frame.split('_')
    
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
        
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame].reset_index()
    ########################## 
        
    len_this_tracking = len(this_tracking)
    
    helmets_df['center_h_p'] = (helmets_df['left']+helmets_df['width']/2).astype(int)
    helmets_df['center_h_m'] = (helmets_df['left']+helmets_df['width']/2).astype(int)*-1
    helmets_df = helmets_df[helmets_df['conf']>CONF_THRE].copy()
    
    
        
    if len(helmets_df) > len_this_tracking:
        helmets_df = helmets_df.tail(len_this_tracking)
    
    ## x軸でソート
    helmets_df_p = helmets_df.sort_values('center_h_p').copy()
    
    ## 逆順でもソート
    helmets_df_m = helmets_df.sort_values('center_h_m').copy()
    
    
    if view == 'Endzone':        
        this_tracking['x'], this_tracking['y'] = this_tracking['y'].copy(), this_tracking['x'].copy()
        
    helmet_center_a2_p = helmets_df_p['center_h_p'].values
    helmet_center_a2_m = helmets_df_m['center_h_m'].values
    

    min_dist_p, min_detete_idx_p,best_angle_p = dist_rot(this_tracking,helmet_center_a2_p,view,DIG_MAX,DIG_STEP,max_iter)    
    min_dist_m, min_detete_idx_m,best_angle_m = dist_rot(this_tracking,helmet_center_a2_m,view,DIG_MAX,DIG_STEP,max_iter)
    
    
    if min_dist_p < min_dist_m:
        min_dist = min_dist_p
        min_detete_idx = min_detete_idx_p
        tgt_df = helmets_df_p
        best_angle = best_angle_p
    else:
        min_dist = min_dist_m
        min_detete_idx = min_detete_idx_m
        tgt_df = helmets_df_m
        best_angle = best_angle_m
    
    
    tgt_df['label'] = min_detete_idx
    tgt_df['best_angle'] = best_angle
    return tgt_df[['video_frame','left','width','top','height','label',"best_angle"]]

In [7]:
submission_df_list = []
df_list = list(helmets.groupby('video_frame'))
df_list = [list(element) + [tracking,paramater] for element in df_list];

In [8]:
from tqdm import tqdm


from lib.multiprocess import exe_multiprocess_ret_as_list

In [9]:

submission_df_list = []
df_list = list(helmets.groupby('video_frame'))
df_list = [ list(element) + [tracking,paramater] for element in df_list];

submission_df_list = exe_multiprocess_ret_as_list(mapping_df,df_list)




sub = pd.concat(submission_df_list)

scorer = NFLAssignmentScorer(labels)
baseline_score = scorer.score(sub)
print(baseline_score)

100%|██████████| 2230/2230 [12:54<00:00,  2.88it/s]


0.35524297579077574
